In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
import ipywidgets as widget
import kaleido  # import para gerar erro caso esteja faltando

from libraries.gvar import *
gvar['ONLINE'] = False

from libraries.version import *
from libraries.drawgraph import *
from libraries.screen import screen_init, button_scr_call

from libraries.set_template import set_template

# Brazilian R$ format
import locale
locale.setlocale(locale.LC_ALL, 'pt_BR')

'pt_BR'

In [2]:
df = {}
getdata_init(df)
init_countries()

bg_transparent = False

def prepare_template(tp_num):
    global save_file_scale, scale, template_num

    if template_num != tp_num:
        template_num = tp_num
        set_template(template_num)
    
    save_file_scale = 3
    if template_codes[template_num] in ['IN_', 'NEC_']:
        save_file_scale = 0.675
    if template_codes[template_num] in ['INB_', 'JP3_']:
        save_file_scale = 1
    if 'size_sli' in globals():
        size_sli.value = save_file_scale

    scale = 0.7
    if template_codes[template_num] in ['INB_']:
        scale = 0.5
    if template_codes[template_num] in ['SBT', 'JP4_']:
        scale = 0.7

    if 'transp_check' in globals():
        if template_codes[template_num] in ['JP_']:
            transp_check.value = True
        else:
            transp_check.value = False

    if 'send_sel' in globals():
        nitems = get_qty_groups()
        send_sel.options = [x+1 for x in range(nitems)]

prepare_template(template_num)

**Loading Exemplo_Barra_Empilhada...
**Loading Exemplo_Barra_Agrupada...
**Loading Ibov_Dolar...
**Loading Ibov_Dolar_Euro...
**Loading RANK_JP...
**Loading IBOPE_JP...
**Loading MONITOR_PIB_FGV_INDICE...
**Loading MONITOR_PIB_FGV_INDICE_MESMO_MES...
**Loading MONITOR_PIB_FGV_INDICE_12M...
**Loading MONITOR_PIB_FGV_INDICE_ANO...
**Loading MONITOR_PIB_FGV_INDICE_ANUAL...
**Loading MONITOR_PIB_FGV_VALOR_CORRENTE...
**Loading Gasolina_Defasagem...
**Loading Diesel_Defasagem...
**Loading MaisRentAbril...
**Loading MaisRentAbril_acum...
**Loading pedidos_demissao...
**Loading Solidaonapandemia...
**Loading Arrecadacao_Federal...
**Loading PIBAMERICANO_BEA...
**Loading Rank_FMI_2022...
**Loading Rank_FMI_2022_pior...
**Loading Rank_FMI_2023...
**Loading Rank_FMI_2023_pior...
**Loading FMI_Previsoes_2022...
**Loading FMI_Previsoes_2023...
**Loading FMI_Previsoes_Abril...
**Loading FMI_Previsoes_4...
**Loading FMI_Previsoes_2022_Agrup...
**Loading FMI_Previsoes_2022atzSet2022...
**Loading FMI_

In [3]:
from datetime import timedelta, datetime

year_prev = 2022
# last day of previous month
today = datetime.today()
# date_ini = datetime(day=1, month=today.month, year=today.year-1) - timedelta(days=1)
# date_end = datetime(day=1, month=today.month, year=today.year  ) - timedelta(days=1)
# date_ini = datetime(day=23, month=2, year=2022)
date_ini = datetime(day=31, month=12, year=2021)
date_end = today

set_date_ini(date_ini)
set_date_end(date_end)
set_dend_type('Close')
set_year_prev(year_prev)

In [4]:
processing_bat = False

def set_videos_check():
    tcheia_check.value, tcheia2_check.value, telao1_check.value, telao2_check.value, telao3_check.value = [False] * 5
    if template_codes[template_num] == 'JP_':
        tcheia_check.value = True
    if template_codes[template_num] in ['JP_', 'JP2_']:
        tcheia2_check.value = True

def process_bat_ini():
    global processing_bat
    processing_bat = True
    gvar['send_message'] = send_txt.value

def process_bat_end():
    global processing_bat
    close_browser()
    processing_bat = False

def process_bat_item(symbol_name, ftype):
    global symbol_old
    debug.value = f'{symbol_name} {ftype} . '
    while True:
        symbol_old = symbol_name
        button_call(ftype)
        if get_some_error():  # Some error in any place
            debug.value += f' [ERROR: {get_some_error()}]'
            time.sleep(10)  # Wait 10s
        else:
            debug.value = ''
            break

def process_bat():
    process_bat_ini()
    df_bat = pd.read_csv('data/config/auto.csv', sep=';')
    for i in range(df_bat.shape[0]):
        symbol_name = asset_name_to_code(df_bat.loc[i,'symbol'])
        ftype = df_bat.loc[i,'type']
        process_bat_item(symbol_name, ftype)
    process_bat_end()

def process_bat_auto():
    process_bat_ini()
    ftype = 'png'
    if tcheia_check.value | tcheia2_check.value | telao1_check.value | telao2_check.value | telao3_check.value:
        ftype = 'vid'
    for symbol_name in list(auto_sel.options):
        if symbol_name.strip() != '':
            process_bat_item(symbol_name, ftype)
    process_bat_end()

In [5]:
# def vbox_msg(widget, content, buffers):
#     debug.value += '[1] '

# def update_point(trace, points, selector):
#     save_error(
#         f'trace: {type(trace)}\n{trace}\n\n'
#         f'points: {type(points)}\n{points}\n\n'
#         f'selector:{type(selector)}\n{selector}\n\n'
#         , open_mode='a')

#     x, y = selector.xrange, selector.yrange
#     gvar['mouse_x'] = x[0]
#     gvar['mouse_y'] = y[0]
#     redraw_symbol()

# def add_mouse(fig):
#     # print(fig.data, type(fig.data), len(fig.data))
#     if len(fig.data) == 0:
#         # Add invisible scatter trace.
#         fig.add_trace(
#             go.Scatter(
#                 x=[0, 800], y=[0, 600],
#                 mode="markers", marker_opacity=0
#             )
#         )

#     scatter = fig.data[-1]
#     scatter.on_click(update_point)
#     scatter.on_selection(update_point)

#     fig.update_layout(modebar_add=['drawrect'])  # 'select'    

In [6]:
symbol_old, symbol_old_real = '', ''

def call_draw_graph(scale, symbol):
    title, subtit, dfont = '', '', ''

    
    if get_txt_digitado():
        print("load_info_digitado")
        load_info_digitado('digitado', symbol=symbol_old_real)
        set_txt_digitado_values(
            symbol_list[symbol]['title'],
            symbol_list[symbol]['subtit'],
            symbol_list[symbol]['dfont'] )
    
    if symbol in symbol_list:
        title  = symbol_list[symbol]['title']
        subtit = symbol_list[symbol]['subtit']
        dfont  = symbol_list[symbol]['dfont']

    gvar['ERROR'] = ''

    ret = draw_graph(scale, symbol, 13,
            title, subtit, dfont, bg_transparent,)

    gerror.value = str(gvar['ERROR'])

    return ret


def update_symbol(symbol, update=True):
    global vbox, fig, scale, symbol_old

    symbol = asset_name_to_code(symbol)

    if symbol == symbol_old and symbol != '':
        return

    if get_txt_digitado() and symbol != symbol_old_real:
        set_txt_digitado(False)
        if symbol_old_real in symbol_list_backup:
            symbol_list[symbol_old_real] = copy.deepcopy(symbol_list_backup[symbol_old_real])
        gvar['dig_mode'] = False
        
    symbol_old = symbol
    symbol_txt.value = symbol

    if get_grapht() and not update:
        return

    fig = call_draw_graph(scale, symbol)

    vbox.children = vbox.children[0:-1] + (fig,)

    # add_mouse(fig)
    # vbox.on_msg(vbox_msg)


def redraw_symbol():
    global symbol_old
    symbol = symbol_old
    symbol_old = ''
    update_symbol(symbol)


def on_submit_symbol(txt):
    if get_grapht():
        global symbol_old
        symbol_old = ''
    # update_symbol(txt.value.upper())
    update_symbol(txt.value)


def get_symbols(group):
    symbol_sel_list = []
    if group == 'PIB':
        symbol_sel_list.append('Div_Liq_PIB_Perc')
        symbol_sel_list.append('Div_Bruta_PIB_Perc')
    for symbol in symbol_list:
        if symbol[:len(group)] == group:
            symbol_sel_list.append(symbol)
    return symbol_sel_list

def on_submit_group(txt):
    group = txt.value.lower()
    symbol_sel_list = []
    for symbol in symbol_list:
        if group in symbol.lower():
            symbol_sel_list.append(symbol)
    symbol_sel.options = symbol_sel_list

def on_trait_group(chg):
    if chg['name'] != 'value':
        return

    group = chg['new']

    if group == '':
        symbol_sel.options = ['']
        return

    if group == 'TUDO':
        symbol_sel.options = list(symbol_list.keys())
        return

    if group == 'OUTROS':
        symbol_sel_list = []
        groups = ['IPP', 'IBC-BR', 'IPC-Fipe', 'ICONF_CONSUM', 'ICONF_IND', 'Viagens_Desp']
        for group in groups:
            symbol_sel_list += get_symbols(group)
        symbol_sel.options = symbol_sel_list
        return

    if group == 'DIGITADOS':
        symbol_sel.options = get_files_created().copy()
        return

    if group == 'TABLE':
        symbol_sel.options = tables
        return

    symbol_sel.options = get_symbols(group)


noupdate = [False]
def on_trait_symbol(chg):
    if noupdate[0]:
        return
    if chg['name'] != 'value':
        return

    # Adiciona symbol a lista de auto
    if auto_check.value:
        noupdate[0] = True
        if chg['new'] not in list(auto_sel.options):
            auto_sel.options = list(auto_sel.options) + [ chg['new'] ]
        auto_sel.value = chg['new']
        noupdate[0] = False
        
    update_symbol(chg['new'], False)


# def on_trait_gtype(chg):
#     pass


def observe_size_sli(chg):
    if chg['name'] == 'value':
        global save_file_scale
        save_file_scale = chg['new']

def button_call(btype):
    global bg_transparent

    btype = btype.upper()

    if btype == 'BAT':
        if auto_check.value:
            process_bat_auto()
        else:
            process_bat()
        return

    if btype == 'SCR':
        screen_init(vbox)
        return

    if btype == 'UPD':
        debug.value = ' Updating (WAIT...)'
        set_update_rank(True)
        redraw_symbol()
        set_update_rank(False)
        debug.value = ''
        return

    if btype == 'DIG':
        global symbol_old_real
        set_txt_digitado(True)
        symbol_old_real = symbol_old
        upd_txt_digitado()
        gvar['dig_mode'] = True
        redraw_symbol()
        return

    # if btype == 'DEL':
    #     del_new_value(symbol_old)
    #     redraw_symbol()
    #     return
    
    if btype not in ['PNG', 'JPG', 'VID']:
        return

    if not processing_bat:
        gvar['send_message'] = send_txt.value
    
    try:
        gvar['send_groups_num'] = int(send_sel.value)
    except:
        gvar['send_groups_num'] = 0

    bg_transparent = transp_check.value

    if btype == 'JPG':
        bg_transparent = False

    if btype == 'VID':
        debug.value += 'Generating Video (WAIT...)'
        bg_transparent = True
    
    fig_save = call_draw_graph(save_file_scale, symbol_old)
    
    filepath = f'graphics/{template_codes[template_num]}_{get_save_file_name()}.{btype.lower()}'
    if btype == 'VID':
        filepath = file_img_to_video
    
    delete_file(filepath)  # Apaga arquivo antigo p/ ficar com data ok
    
    print("gravando...")
    pio.write_image(fig_save, filepath)
    print("gravado.")
    
    send_to_clipboard(filepath)
    
    if btype in ['PNG', 'JPG']:
        copy_img_to_remote(filepath)
        send_by_telegram(filepath)
    
    if btype == 'VID':
        if tcheia_check.value:  graph_create_video()
        if tcheia2_check.value: graph_create_video(pos='CHEIA2')
        if telao1_check.value: graph_create_video(pos='TELAO_ESQ')
        if telao2_check.value: graph_create_video(pos='TELAO_MEIO')
        if telao3_check.value: graph_create_video(pos='TELAO_DIR')
        bg_transparent = False
        debug.value = ''
        delete_file(filepath)  # Apaga imagem usada para gerar o vídeo

    if not processing_bat:
        close_browser()

    bg_transparent = False

def on_button_clicked(btn):
    button_call(btn.description)

def observe_template_sel(chg):
    if chg['name'] == 'value':
        set_config('template_num', chg['new'])
        prepare_template(int(chg['new']))
        redraw_symbol()
        set_videos_check()


def on_trait_auto(chg):
    if noupdate[0]: return
    if chg['name'] != 'value': return
    if chg['new'].strip() == '': return

    noupdate[0] = True
    new_list = list(auto_sel.options)
    new_list.remove(chg['new'])
    auto_sel.options = new_list
    # auto_sel.value = (new_list + ['', ''])[1]
    auto_sel.value = ''
    noupdate[0] = False
    

def check_observe(chg):
    if chg['name'] != 'value': return

    caller = chg['owner'].description.lower()

    if caller == 'auto':
        if chg['new']:
            auto_sel.options=['']
            auto_sel.layout.visibility = 'visible'
        else:
            auto_sel.layout.visibility = 'hidden'

    if caller == 'send':
        gvar['send_by_telegram'] = chg['new']


def grapht_observe(chg):
    # save_error(f'{chg}\n', open_mode='a')
    if chg['name'] == 'value':
        if chg['owner'].description.lower() == 'val':  # grapht_val_check
            gvar['grapht_val'] = chg['new']
            if not get_grapht():
                if chg['new']:
                    grapht_check.value = True  # this change forces call to 'grapht_observe' again
                return
        else:                                           # grapht_check
            set_grapht(chg['new'])

        update_symbol('')
        noupdate[0] = True
        group_sel.value = ''
        symbol_sel.value = ''
        noupdate[0] = False

def year_prev_set(chg):
    if chg['name'] == 'value':
        set_year_prev(chg['new'])
        redraw_symbol()

def date_ini_set(chg):
    if chg['name'] == 'value':
        set_date_ini(datetime.combine(chg['new'], datetime.min.time()))
        noupdate[0] = True
        period_sel.value = bt_text[8]
        noupdate[0] = False
        set_g001_period(period_sel.value)
        redraw_symbol()

def date_end_set(chg):
    if chg['name'] == 'value':
        set_date_end(datetime.combine(chg['new'], datetime.min.time()))
        noupdate[0] = True
        period_sel.value = bt_text[8]
        noupdate[0] = False
        set_g001_period(period_sel.value)
        redraw_symbol()

def dend_type_set(chg):
    if chg['name'] == 'value':
        set_dend_type(chg['new'])
        redraw_symbol()

def period_set(chg):
    if noupdate[0]:
        return
    if chg['name'] == 'value':
        set_g001_period(chg['new'])
        redraw_symbol()

def country_set(chg):
    if chg['name'] == 'value':
        set_country_filter(chg['new'])
        redraw_symbol()

# def on_submit_new_value(txt):
#     add_new_value(symbol_old, txt)
#     redraw_symbol()


fig = go.FigureWidget()

default_symbol = 'teste'
symbol_txt = widget.Text(
    value=default_symbol, placeholder='Escolha ítem',
    layout=widget.Layout(width='120px', grid_gap='20px'),
    indent=False,
    # continuous_update=False,
    # description='String:'
)
symbol_txt.on_submit(on_submit_symbol)


group_txt = widget.Text(
    value='', placeholder='procurar ...',
    layout=widget.Layout(width='120px', grid_gap='20px'),
    indent=False,
)
group_txt.on_submit(on_submit_group)

group_sel = widget.Dropdown(
    options=['TUDO', 'IPCA', 'INPC', 'IGP', 'INCC',
            'FOCUS', 'GRAPH', 'RANK', 'TABLE',
            'PIB', 'PROD_IND', 'VOL_SERV', 'VENDAS_COM', 'PNAD', 'CAGED',
            'OUTROS', 'DIGITADOS', ''],
    value='TUDO', # description='Group:',
    # indent=False,
    layout=widget.Layout(width='120px'),
)
group_sel.observe(on_trait_group)


# symbol_sel = widget.Select(
symbol_sel = widget.Dropdown(
    options=list(symbol_list.keys()),
    value='IPCA', # description='Sel:',
    # indent=False, # rows=1,
    layout=widget.Layout(width='200px'),
)
symbol_sel.observe(on_trait_symbol)


gtype_sel = widget.Dropdown(
    options=['MENSAL', 'MESMO MÊS', '12 MESES', 'NO ANO', 'ANUAL'],
    value='MENSAL', layout=widget.Layout(width='100px'),
)
# gtype_sel.observe(on_trait_gtype)


size_sli = widget.FloatSlider(
    # description='Tam.:',
    layout=widget.Layout(width='150px'),
    value=save_file_scale, min=0.5, max=4.0, step=0.25,
    orientation='horizontal',
    continuous_update=False,
    # readout=True, readout_format='.1f',
)
size_sli.observe(observe_size_sli)


bt_lay = layout=widget.Layout(width='40px')

save_bt = widget.Button(description='png', layout=widget.Layout(width='42px'))
save_bt.on_click(on_button_clicked)

jpg_bt = widget.Button(description='jpg', layout=bt_lay)
jpg_bt.on_click(on_button_clicked)

vid_bt = widget.Button(description='vid', layout=bt_lay)
vid_bt.on_click(on_button_clicked)

bat_bt = widget.Button(description='bat', layout=bt_lay)
bat_bt.on_click(on_button_clicked)

dig_bt = widget.Button(description='dig', layout=bt_lay)
dig_bt.on_click(on_button_clicked)

upd_bt = widget.Button(description='upd', layout=widget.Layout(width='42px'))
upd_bt.on_click(on_button_clicked)

scr_bt = widget.Button(description='scr', layout=widget.Layout(width='42px'))
scr_bt.on_click(on_button_clicked)


template_sel = widget.Dropdown(
    options=list(range(1, len(template_codes))), value=template_num, 
    # description='Style:', # indent=False,
    layout=widget.Layout(width='40px'),
)
template_sel.observe(observe_template_sel)


grapht_check = widget.Checkbox(
    value=False, description='GT', indent=False,
    layout=widget.Layout(width='40px'),
)
grapht_check.observe(grapht_observe)

grapht_val_check = widget.Checkbox(
    value=False, description='Val', indent=False,
    layout=widget.Layout(width='50px'),
)
grapht_val_check.observe(grapht_observe)


send_check = widget.Checkbox(
    value=False, description='Send', indent=False,
    layout=widget.Layout(width='50px'),
)
send_check.observe(check_observe)

send_sel = widget.Dropdown(
    options=[], indent=False,
    layout=widget.Layout(width='40px'),
)

send_txt = widget.Text(
    value='', placeholder='Mensgagem para envio',
    layout=widget.Layout(width='200px', grid_gap='20px'),
    indent=False,
)

auto_check = widget.Checkbox(
    value=False, description='Auto', indent=False,
    layout=widget.Layout(width='50px'),
)
auto_check.observe(check_observe)

auto_sel = widget.Dropdown(
    options=[], indent=False,
    layout=widget.Layout(width='200px', visibility='hidden'),
)
auto_sel.observe(on_trait_auto)

transp_check = widget.Checkbox(
    value=False, description='Tra', indent=False,
    layout=widget.Layout(width='50px'),
)
# transp_check.observe(transp_observe)
if template_codes[template_num] in ['JP_']:
    transp_check.value = True


tcheia_check = widget.Checkbox(value=False, indent=False, layout=widget.Layout(width='10px'),)
tcheia2_check = widget.Checkbox(value=True, indent=False, layout=widget.Layout(width='10px'),)
telao1_check = widget.Checkbox(value=False, indent=False, layout=widget.Layout(width='10px'),)
telao2_check = widget.Checkbox(value=False, indent=False, layout=widget.Layout(width='10px'),)
telao3_check = widget.Checkbox(value=False, indent=False, layout=widget.Layout(width='10px'),)

year_prev_sel = widget.Dropdown(
    options=list(range(2020, 2026)),
    value=year_prev,
    layout=widget.Layout(width='70px'),
)
year_prev_sel.observe(year_prev_set)

date_ini_sel = widget.DatePicker(
    value = date_ini, continuous_update=False,
    layout=widget.Layout(width='160px'),
)
date_ini_sel.observe(date_ini_set)

date_end_sel = widget.DatePicker(
    value=date_end, continuous_update=False,
    layout=widget.Layout(width='160px'),
)
date_end_sel.observe(date_end_set)

dend_type_sel = widget.Dropdown(
    # options=['Fech', 'Min', 'Max', 'Aber'],
    options=['Close', 'Low', 'High', 'Open'],
    layout=widget.Layout(width='70px'),
)
dend_type_sel.observe(dend_type_set)

period_sel = widget.Dropdown(
    options=bt_text,
    layout=widget.Layout(width='60px'),
)
period_sel.observe(period_set)

country_sel = widget.Dropdown(
    options=country_filters,
    layout=widget.Layout(width='120px'),
)
country_sel.observe(country_set)


# Campo para adição de novos valores no final do arquivo
# new_value_txt = widget.Text(
#     value='', placeholder='Novo valor ?',
#     layout=widget.Layout(width='120px', grid_gap='20px'),
#     indent=False,
# )
# new_value_txt.on_submit(on_submit_new_value)


def txt_layout(placeholder, width):
    return widget.Text(value='', placeholder=placeholder,
        layout=widget.Layout(width=f'{width}px', grid_gap='20px'), indent=False,)

# Dados para o digitado.csv
title_txt       = txt_layout('Título',      150)
title_fs_txt    = txt_layout('tam',          40)
subtit_txt      = txt_layout('Subtítulo',   150)
subtit_fs_txt   = txt_layout('tam',          40)
dfont_txt       = txt_layout('Fonte Dados', 100)
decimal_txt     = txt_layout('Dec',          40)
quant_txt       = txt_layout('barN',         50)
blk_dx_txt      = txt_layout('barD',         50)
blk_x_txt       = txt_layout('barX',         50)
blk_y_txt       = txt_layout('barY',         50)
blk_w_txt       = txt_layout('barW',         50)
blk_h_txt       = txt_layout('barH',         50)
bar_val_fs_txt  = txt_layout('fontV',        56)
xaxis_fs_txt    = txt_layout('fontX',        56)
yref_txt        = txt_layout('Yref',         50)
ylines_txt      = txt_layout('Linhas Y',    100)  # acrescentar yref se não houver
xstep_txt       = txt_layout('stepX',        56)

debug = widget.Label(value = '')
gerror = widget.Label(value = '')

size_lbl = widget.Label(value = ' \xa0 Tam.:')
template_lbl = widget.Label(value = ' \xa0 Style:')
version = widget.Label(value = f' \xa0 v{VERSION}')
videochk_lbl = widget.Label(value = ' \xa0 12LMR:')


set_view_fields(date_ini_sel, date_end_sel, dend_type_sel, year_prev_sel, period_sel, country_sel, upd_bt, debug)

years_enable(False)
set_videos_check()

gvar['ERROR'] = ''
gvar['grapht_val'] = False


# O notebook não pára após erro, então isso não faz sentido
# Precisa ver como parar o Notebook
# working = widget.Label(value = '-')
# show_working_timer(working)


box_style = ("<style>.box_bg_style{border:4px solid black; background-color:#999999;}</style>")
vbox = widget.VBox([
        widget.HTML(box_style),
        # widget.HTML('<style>.widget-text{width:50px;}</style>'),
        widget.HBox([symbol_txt,
            group_sel, group_txt, symbol_sel, # gtype_sel,
            size_lbl, size_sli,
            transp_check, save_bt, jpg_bt, vid_bt,
            videochk_lbl,
            tcheia_check, tcheia2_check, telao1_check, telao2_check, telao3_check,
            # template_lbl,
            template_sel, version, # working,
        ]),
        widget.HBox([scr_bt, bat_bt, dig_bt, grapht_check, grapht_val_check, auto_check, auto_sel,
            year_prev_sel, date_ini_sel, date_end_sel, dend_type_sel, period_sel, country_sel,
            upd_bt,  # new_value_txt,
            gerror, debug,
        ]),
        widget.HBox([send_check, send_sel, send_txt,
        ]),
        widget.HBox([title_txt,title_fs_txt,  subtit_txt,subtit_fs_txt,  dfont_txt,
            decimal_txt, quant_txt, blk_dx_txt, blk_x_txt, blk_y_txt, blk_w_txt, blk_h_txt,
            bar_val_fs_txt, xaxis_fs_txt,
            yref_txt, ylines_txt, xstep_txt,
        ]),
        fig
    ])

vbox.add_class('box_bg_style');


on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').



on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').




**Loading BC_US...
BC_US
entrou em BC
            data valor
0     16/12/2008  0.25
1     17/12/2008  0.25
2     18/12/2008  0.25
3     19/12/2008  0.25
4     20/12/2008  0.25
...          ...   ...
5418  17/10/2023  5.50
5419  18/10/2023  5.50
5420  19/10/2023  5.50
5421  20/10/2023  5.50
5422  21/10/2023  5.50

[5423 rows x 2 columns]
**Loading BC_EUR...
BC_EUR
entrou em BC
            data valor
0     01/01/1999   2.0
1     02/01/1999   2.0
2     03/01/1999   2.0
3     04/01/1999  2.75
4     05/01/1999  2.75
...          ...   ...
9054  16/10/2023   4.0
9055  17/10/2023   4.0
9056  18/10/2023   4.0
9057  19/10/2023   4.0
9058  20/10/2023   4.0

[9059 rows x 2 columns]


In [7]:
symbol_old = ''
update_symbol('IPCA')

# update_symbol('GRAPH_IBOV_x_CDI')
# update_symbol('pizza')
# update_symbol('PNAD_Desemprego_Perc_Linha')
# update_symbol('IronOre')
                
# update_symbol('PROJECOES_Table')
# update_symbol('IBC-BR_ANUAL')
# update_symbol('RANK_Juros_Reais')
# update_symbol('RANK_ALL_USD_Pior')
# update_symbol('IBOV')
# button_call('BAT')
# button_call('SCR')
# button_scr_call('VID')

# set_grapht(True)
# update_symbol('inv_com_iron')
# update_symbol('IronOre')
# gvar['ipca_on'] = True
# update_symbol('gasol')
# update_symbol('etanol')
# update_symbol('diesel')
# update_symbol('brentr')

# update_symbol('IBOV')
# button_call('DIG')

# set_grapht(True)
# update_symbol('PRE20220128')
# update_symbol('PRE20210104')

def upd_txt_digitado():
    fp = open('data/digitado/digitado.csv', 'w', encoding='utf8')

    # if gtype_sel.value == '12 MESES':
    #     fp.write('subtit;Variação acumulada em 12 meses, em %\n')
    #     fp.write('group;12\n')
    #     fp.write('ylines;0\n')

    tsuf = ''
    if template_codes[template_num] == 'JP3_':
        tsuf = template_sufix[template_num]
    
    fp.write(f'title{tsuf};{title_txt.value}\n')
    fp.write(f'title_fs{tsuf};{title_fs_txt.value}\n')
    fp.write(f'subtit{tsuf};{subtit_txt.value}\n')
    fp.write(f'subtit_fs{tsuf};{subtit_fs_txt.value}\n')
    fp.write(f'dfont{tsuf};{dfont_txt.value}\n')
    fp.write(f'decimal{tsuf};{decimal_txt.value}\n')
    fp.write(f'quant{tsuf};{quant_txt.value}\n')
    fp.write(f'blk_dx{tsuf};{blk_dx_txt.value}\n')
    fp.write(f'blk_x{tsuf};{blk_x_txt.value}\n')
    fp.write(f'blk_y{tsuf};{blk_y_txt.value}\n')
    fp.write(f'blk_w{tsuf};{blk_w_txt.value}\n')
    fp.write(f'blk_h{tsuf};{blk_h_txt.value}\n')
    fp.write(f'bar_val_fs{tsuf};{bar_val_fs_txt.value}\n')
    fp.write(f'xaxis_fs{tsuf};{xaxis_fs_txt.value}\n')
    fp.write(f'yref{tsuf};{yref_txt.value}\n')
    fp.write(f'ylines{tsuf};{ylines_txt.value}\n')
    fp.write(f'xstep{tsuf};{xstep_txt.value}\n')

    fp.close()

if gvar['ERROR'] != '':
    print(gvar['ERROR'])

prepare_template(template_num)

vbox

**Loading IPCA...
IBGE_1737,63,0,0
1737
63
None


In [8]:
import requests

url = f'https://data-api.ecb.europa.eu/service/data/EXR/M.USD.EUR.SP00.A?format=json'
requests.get(url).text


'{"_links":{"self":{"href":"http://data-api.ecb.europa.eu/service/data/EXR/M.USD.EUR.SP00.A"},"alternate":[{"name":"SDMX-ML 2.1 Generic Data","title":"SDMX-ML 2.1 Generic Data (default)","href":"http://data-api.ecb.europa.eu/service/data/EXR/M.USD.EUR.SP00.A","type":"application/vnd.sdmx.genericdata+xml;version=2.1"},{"name":"SDMX-ML 2.1 Structure Specific Data","title":"SDMX-ML 2.1 Structure Specific Data format","href":"http://data-api.ecb.europa.eu/service/data/EXR/M.USD.EUR.SP00.A","type":"application/vnd.sdmx.structurespecificdata+xml;version=2.1"},{"name":"SDMX-JSON","title":"SDMX-JSON Data format","href":"http://data-api.ecb.europa.eu/service/data/EXR/M.USD.EUR.SP00.A","type":"application/vnd.sdmx.data+json;version=1.0.0-wd"},{"name":"CSV","title":"SDMX-JSON Data format","href":"http://data-api.ecb.europa.eu/service/data/EXR/M.USD.EUR.SP00.A","type":"text/csv"},{"name":"CSV Pivot","title":"CSV format, optimised for pivot tables","href":"http://data-api.ecb.europa.eu/service/data

In [74]:
# Codigo para Media dos Mais Caros e Mais Baratos ANP em SP e RJ
# Realizado: Media ponderada para cada combustivel de cada estado

comb = ['diesel_rj', 'diesel_sp', 'diesel_s10_rj', 'diesel_s10_sp', 
        'etanol_rj', 'etanol_sp', 'gasol_adit_rj', 'gasol_adit_sp', 
        'gasolina_sp', 'gasolina_rj', 'glp_rj', 'glp_sp', 'gnv_rj', 'gnv_sp']

data = {'Diesel': [], 'Diesel S10': [], 'Etanol': [], 'Gasolina Adit': [], 'Gasolina': [], 'GLP': [], 'GNV': []}
data_keys = list(data.keys())

url = 'https://www.primetalkdata.com/datacenter/combustiveis/'

for type in ['mais_caro/', 'mais_barato/']:
    date = requests.get(url + type).text.split('<a href="')[-1][:10]
    
    for idx in range(len(comb)):
        auxDF = pd.DataFrame([elem.split(',') for elem in requests.get(url + type + date + '_' + comb[idx] + '.csv').text.split('\r\n')])
        auxDF.columns = auxDF.iloc[0]
        auxDF.drop(auxDF.index[0], inplace=True)
        auxDF.dropna(inplace=True)

        val = auxDF[auxDF['DATA INICIAL'] == auxDF['DATA INICIAL'].iloc[-1]].iloc[:,-1:].astype(float).values
        wgt = auxDF[auxDF['DATA INICIAL'] == auxDF['DATA INICIAL'].iloc[-1]].iloc[:,-3:-2].astype(int).values

        data[data_keys[idx//2]].append(round(np.average(val, weights=wgt), 2))

pd.DataFrame(data, index=pd.MultiIndex.from_tuples([('RJ', 'Mais Caro'), ('RJ', 'Mais Barato'), ('SP', 'Mais Caro'), ('SP', 'Mais Barato')]))


Diesel  Diesel S10  Etanol  Gasolina Adit  Gasolina     GLP  \
RJ Mais Caro      6.41        6.69    4.79           6.40      6.05  116.93   
   Mais Barato    6.49        6.61    3.99           6.43      6.33  114.55   
SP Mais Caro      5.75        5.87    3.86           5.56      5.28   85.01   
   Mais Barato    5.67        5.88    3.06           5.44      5.48   85.56   

                 GNV  
RJ Mais Caro    6.04  
   Mais Barato  4.83  
SP Mais Caro    4.26  
   Mais Barato  3.90

In [99]:
# Codigo para 5 Maiores Altas e 5 Maiores Quedas pela CEAGESP
# Realizado: 5 maiores altas e baixas percentuais entre todos os setores 

url = 'https://www.primetalkdata.com/datacenter/ceagesp/2023/'
date = requests.get(url).text.split('<a href="')[-1][:10]

data = {'Tipo': [], 'Setor': [], 'Variação': [], 'Item': [], 'Percentual': []}

for type in ['alta', 'baixa']:
    resp = requests.get(url + date + '_ceagesp.csv')
    resp.encoding = resp.apparent_encoding
    auxDF = pd.DataFrame([elem.split(',') for elem in resp.text.split('\r\n')])
    auxDF.columns = auxDF.iloc[0]
    auxDF.drop(auxDF.index[0], inplace=True)
    auxDF.dropna(inplace=True)

    auxDF['pct_' + type] = pd.to_numeric(auxDF['pct_' + type], errors='coerce')
    auxDF = auxDF.sort_values('pct_' + type, ascending=(type=='baixa')).head(5)

    data[]
    data['Setor'].extend(auxDF.setor.values)
    data['Variação'].extend(auxDF.variacao.values)
    data['Item'].extend(auxDF['top_' + type + 's'].values)
    data['Percentual'].extend(auxDF['pct_' + type].values) 

pd.DataFrame(data, index=pd.MultiIndex.from_tuples([('Maiores Altas'), ('Maiores Altas'), ('Maiores Altas'), ('Maiores Altas'), ('Maiores Altas'), 
                                                    ('Maiores Baixas'), ('Maiores Baixas'), ('Maiores Baixas'), ('Maiores Baixas'), ('Maiores Baixas')]))

Setor Variação              Item  Percentual
M a i o r e s   A l t a s NaN  VERDURAS    11.33             SALSA      122.15
                          NaN   LEGUMES    16.20      TOMATE CAQUI       84.03
                          NaN    FRUTAS     2.96       LIMÃO TAITI       74.47
                          NaN  VERDURAS    17.25          RABANETE       71.15
                          NaN  VERDURAS    21.15           COENTRO       70.91
                B a i x a s    PESCADOS   -10.03    SARDINHA LAGES      -58.89
                          s    VERDURAS   -10.21           COENTRO      -56.97
                          s    VERDURAS   -13.00             SALSA      -53.50
                          s     LEGUMES    -0.88  PIMENTÃO AMARELO      -49.70
                          s    PESCADOS   -10.03      PEROÁ BRANCO      -48.70

,,setor,variacao,top_altas,pct_alta,top_baixas,pct_baixa
187,2023-02-01,VERDURAS,11.33,SALSA,122.15,ALFACE CRESPA,-11.76
131,2023-04-01,LEGUMES,16.20,TOMATE CAQUI,84.03,ERVILHA TORTA,-27.38
26,2023-08-01,FRUTAS,2.96,LIMÃO TAITI,74.47,MAMÃO HAVAÍ,-46.31
212,2023-01-01,VERDURAS,17.25,RABANETE,71.15,MILHO VERDE,-18.94
162,2023-03-01,VERDURAS,21.15,COENTRO,70.91,REPOLHO LISO,-5.53
